In [71]:
import re 
import nltk
import pickle
import random
import unicodedata
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from tqdm import trange
from itertools import product
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

nltk.download('stopwords')
STOPWORDS_ES = stopwords.words("spanish")
STOPWORDS_EN = stopwords.words('english')

EXPERIMENTAR = False

[nltk_data] Downloading package stopwords to /home/luis-
[nltk_data]     beto/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<h1>MultiLayerPerceptron Class</h1>

In [72]:
class MultiLayerPerceptron:
    def __init__(self, 
                 num_entradas, 
                 num_neuronas_ocultas, 
                 num_salidas, epochs, 
                 inicialitazion_function, 
                 activation_function, 
                 activation_derivative, 
                 loss_function, 
                 normalization_function=None, 
                 batch_size=128, 
                 learning_rate=0.2, 
                 stop_error=0.02,
                 rango_epocas=5,
                 umbral_mejora=0.003):

        self.epochs = epochs
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.initialize = inicialitazion_function
        self.activate = activation_function
        self.activation_derivative = activation_derivative
        self.loss_function = loss_function
        self.normalize = normalization_function
        self.stop_error = stop_error
        self.rango_epocas = rango_epocas
        self.umbral_mejora = umbral_mejora
        
        # Inicializar
        self.W1 = self.initialize(num_neuronas_ocultas, num_entradas)
        self.B1 = np.zeros((1, num_neuronas_ocultas), dtype=np.float64)
        self.W2 = self.initialize(num_salidas, num_neuronas_ocultas)
        self.B2 = np.zeros((1, num_salidas), dtype=np.float64)

    def create_minibatches(self, X, y, batch_size):
        """
        Genera los lotes de datos (batchs) de acuerdo al parámetro batch_size de forma aleatoria para el procesamiento. 
        """
        n_samples = X.shape[0]
        indices = np.random.permutation(n_samples)  # Mezcla los índices aleatoriamente
        X_shuffled, y_shuffled = X[indices], y[indices]  # Reordena X e y según los índices aleatorios
        
        # Divide los datos en minibatches
        for X_batch, y_batch in zip(np.array_split(X_shuffled, np.ceil(n_samples / batch_size)), 
                                    np.array_split(y_shuffled, np.ceil(n_samples / batch_size))):
            yield X_batch, y_batch

    def forward(self, X):
        """
        Realiza el Forward pass
        """
        self.X = X
        self.z_c1 = self.X @ self.W1.T + self.B1
        self.B1 = np.sum(self.B1, axis=0, keepdims=True)
        self.a_c1 = self.activate(self.z_c1)

        self.z_c2 = self.a_c1 @ self.W2.T + self.B2
        self.B2 = np.sum(self.B2 , axis=0, keepdims=True)
        self.y_pred = self.activate(self.z_c2)
        return self.y_pred 

    def backward(self, y):
        """ 
        Realiza la Backpropagation
        """
        self.dE_dy_pred = self.y_pred - y
        self.dy_pred_dz_c2 = self.activation_derivative(self.y_pred)
        
        self.delta_c2 = self.dE_dy_pred * self.dy_pred_dz_c2
        self.grad_c2 = self.delta_c2.T @ self.a_c1

        self.delta_c1 = self.delta_c2 @ self.W2 * self.activation_derivative(self.a_c1)
        self.grad_c1 = self.delta_c1.T @ self.X

    def update(self):
        """
        Re-ajusta los parámetros de la red
        """
        self.W2 =  self.W2 -  self.learning_rate * self.grad_c2
        self.B2 =  self.B2 - self.learning_rate * np.sum(self.delta_c2, axis=0, keepdims=True)
        
        self.W1 = self.W1 - self.learning_rate * self.grad_c1
        self.B1 = self.B1 - self.learning_rate * np.sum(self.delta_c1, axis=0, keepdims=True)

    def predict(self, X, y):
        """
        Predice como un clasificador binario
        """
        self.y_pred = self.forward(X)
        self.y_pred = np.where(self.y_pred >= 0.5, 1, 0)
        return self.y_pred 

    def train(self, X, Y):
        """
        Entrena a la red neuronal con las features X y el target Y
        1.- Forward Pass
        2.- Cálculo del error
        3.- Backpropagation
        4.- Actualización de parámetros
        """
        self.epochs_error = []
        self.X = X
        for epoch in trange(self.epochs, desc="Entrenando"):
            num_batch = 0
            epoch_error  = 0
            for X_batch, y_batch in self.create_minibatches(X, Y, self.batch_size):
                self.y_pred = self.forward(X_batch)
                error = self.loss_function(self.y_pred, y_batch)
                epoch_error += error    
                self.backward(y_batch)
                self.update()
                num_batch += 1
            self.epochs_error.append(epoch_error/num_batch)
            
            #Criterio de paro
            if epoch_error/num_batch < self.stop_error:
                print(f"Paro por error mínimo en época {epoch}")
                self.epochs = epoch + 1
                break
            elif epoch % self.rango_epocas == 0 and len(self.epochs_error) > 2 * self.rango_epocas:
                rango_anterior = np.mean(self.epochs_error[-2 * self.rango_epocas : -self.rango_epocas])
                rango_actual = np.mean(self.epochs_error[-self.rango_epocas:])
                mejora = rango_anterior - rango_actual
                if mejora < self.umbral_mejora:
                    print(f"Paro por falta de mejora en época {epoch}")
                    self.epochs = epoch + 1
                    break

<h1>Estrategias de ejecución</h1>

In [73]:
# -----------------------
# Funciones de activación
# -----------------------
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

# -----------------------
# Funciones de error
# -----------------------
def loss_function_MSE(y_pred, y):
    error = np.divide((y_pred - y) ** 2, 2 * y.shape[0])
    total_error =  np.sum(error)
    return total_error

# ---------------------------
# Funciones de inicialización
# ---------------------------
def xavier_initialization(input_size, output_size):
    x = np.sqrt(6 / (input_size + output_size))
    array = np.random.uniform(-x, x, (input_size, output_size))
    return np.float64(array)
    
def normal_distribution_initialization(input_size, output_size):
    array = np.random.normal(0, 0.1, (input_size, output_size))
    return np.float64(array)

# -----------------------------
# Funciones de preprocesamiento
# -----------------------------
def normaliza_texto(input_str,
                    punct=False,
                    accents=False,
                    num=False,
                    max_dup=2):
    """
        punct=False (elimina la puntuación, True deja intacta la puntuación)
        accents=False (elimina los acentos, True deja intactos los acentos)
        num= False (elimina los números, True deja intactos los acentos)
        max_dup=2 (número máximo de símbolos duplicados de forma consecutiva, rrrrr => rr)
    """
    PUNCTUACTION = ";:,.\\-\"'/"
    SYMBOLS = "()[]¿?¡!{}~<>|"
    NUMBERS= "0123456789"
    SKIP_SYMBOLS = set(PUNCTUACTION + SYMBOLS)
    SKIP_SYMBOLS_AND_SPACES = set(PUNCTUACTION + SYMBOLS + '\t\n\r ')

    nfkd_f = unicodedata.normalize('NFKD', input_str)
    n_str = []
    c_prev = ''
    cc_prev = 0
    for c in nfkd_f:
        if not num:
            if c in NUMBERS:
                continue
        if not punct:
            if c in SKIP_SYMBOLS:
                continue
        if not accents and unicodedata.combining(c):
            continue
        if c_prev == c:
            cc_prev += 1
            if cc_prev >= max_dup:
                continue
        else:
            cc_prev = 0
        n_str.append(c)
        c_prev = c
    texto = unicodedata.normalize('NFKD', "".join(n_str))
    texto = re.sub(r'(\s)+', r' ', texto.strip(), flags=re.IGNORECASE)
    return texto

def eliminar_stopwords(texto: str, idioma):
    if idioma == 'ES':
        tokens = [t for t in texto.split() if t not in STOPWORDS_ES]
        return ' '.join(tokens)
    elif idioma == 'EN':
        tokens = [t for t in texto.split() if t not in STOPWORDS_EN]
        return ' '.join(tokens)
    
def aplicar_stemming(texto: str, idioma):
    if idioma == 'ES':
        stemmer = SnowballStemmer("spanish")
    elif idioma == 'EN':
        stemmer = SnowballStemmer("english")
    tokens = [stemmer.stem(t) for t in texto.split()]
    return ' '.join(tokens)

# Esta función se usa para pasar al vectorizador,
# Ya que no se puede pasar mas de 1 parametro 
# Con una función interna se puede acceder a mas de 1 parámetro
def preprocesar(preprocesamientos, idioma):
    def aplicar_preprocesamiento(texto):
        if preprocesamientos[0] != None:
            texto = preprocesamientos[0](texto)
        if preprocesamientos[1] != None:
            texto = preprocesamientos[1](texto, idioma)
        if preprocesamientos[2] != None:
            texto = preprocesamientos[2](texto, idioma)
        return texto
    return aplicar_preprocesamiento
    

<h1>Experimentación</h1>
<h2>Funciones para el ciclo de experimentación</h2>

In [74]:
def tokenizar(textos, vectorizador, ngram_range, preprocesamientos, idioma):
    if vectorizador == 'TF':
        vec = CountVectorizer(analyzer='word', 
                              preprocessor=preprocesar(preprocesamientos, idioma),
                              ngram_range=ngram_range)
    elif vectorizador == 'TF-IDF':
        vec = TfidfVectorizer(analyzer='word', 
                              preprocessor=preprocesar(preprocesamientos, idioma),
                              ngram_range=ngram_range)
    X = vec.fit_transform(textos)
    return vec, X

def guardar_modelos(tupla, filename):
    file = open(filename, 'ab')
    pickle.dump(tupla, file)                    
    file.close()

def graficar(diccionario, mlp):
    """
    Grafica y guarda la imagen del error conforme a las épocas durante 
    el entrenamiento de la red neuronal
    """
    filename = f'{diccionario['Dataset']}_'
    filename += f'{diccionario['Pesado']}_'
    filename += f'{diccionario['N-gramas']}_'
    filename += f'{diccionario['Preprocesamiento']}_'
    filename += f'{diccionario['Initialization']}_'
    filename += f'{diccionario['Hidden-layer-size']}_'
    filename += f'lr{int(diccionario['Learning-rate']*100)}_'
    filename += f'{diccionario['Batch-size']}.png'
    caracteristicas = f'Dataset: {diccionario['Dataset']}, '
    caracteristicas += f'Método de pesado: {diccionario['Pesado']}, '
    caracteristicas += f'Representación de términos: {diccionario['N-gramas']}, '
    caracteristicas += f'Preprocesamiento de datos: {diccionario['Preprocesamiento']}\n'
    caracteristicas += f'Función de Inicialización: {diccionario['Initialization']}, '
    caracteristicas += f'Neuronas ocultas: {diccionario['Hidden-layer-size']}, '
    caracteristicas += f'Learning Rate: {diccionario['Learning-rate']}, '
    caracteristicas += f'Batch Size: {diccionario['Batch-size']}'
    
    plt.figure(figsize=(16, 9))
    plt.title(f'Reconocimiento de discurso de odio {diccionario['Dataset']}')
    plt.figtext(0.13, 0.02, caracteristicas)
    plt.plot(np.arange(mlp.epochs), mlp.epochs_error, color='aqua', linestyle='-', linewidth=1, label='MSE')
    plt.xlabel('Época')
    plt.ylabel('MSE')
    plt.grid(True)
    plt.savefig(f'./graficas/{filename}')
    plt.close()

def test_modelo(modelo: MultiLayerPerceptron, vectorizador: CountVectorizer | TfidfVectorizer, X, Y_test):
    X_vec = vectorizador.transform(X)
    y_pred = modelo.predict(X_vec,Y_test)
    
    a = accuracy_score(Y_test, y_pred)
    f1 = f1_score(Y_test, y_pred, average='macro')
    pr = precision_score(Y_test, y_pred, average='macro')
    rc = recall_score(Y_test, y_pred, average='macro')
    print(f'F1-score: {f1}')
    print(f'Precision: {pr}')
    print(f'Recall: {rc}')
    print(f'Accuracy: {a}')
    return a, f1, pr, rc

<h2>Datos de entrenamiento y prueba</h2>

In [75]:
if EXPERIMENTAR:
    # --------------------------
    # Documentos en inglés
    # --------------------------
    docs = pd.read_json('./datasets/hateval_en_train.json', lines=True)
    textos_en_train = docs['text']
    Y_en_train = docs['klass']
    Y_en_train = Y_en_train.to_numpy().reshape(-1, 1)

    docs = pd.read_json('./datasets/hateval_en_test.json', lines=True)
    textos_en_test = docs['text']
    Y_en_test = docs['klass']
    Y_en_test = Y_en_test.to_numpy().reshape(-1, 1)

    docs = pd.read_json('./datasets/hateval_en_all.json', lines=True)
    textos_en_all = docs['text']
    Y_en_all = docs['klass']
    Y_en_all = Y_en_all.to_numpy().reshape(-1, 1)

    # --------------------------
    # Documentos en español
    # --------------------------
    docs = pd.read_json('./datasets/hateval_es_train.json', lines=True)
    textos_es_train = docs['text']
    Y_es_train = docs['klass']
    Y_es_train = Y_es_train.to_numpy().reshape(-1, 1)

    docs = pd.read_json('./datasets/hateval_es_test.json', lines=True)
    textos_es_test = docs['text']
    Y_es_test = docs['klass']
    Y_es_test = Y_es_test.to_numpy().reshape(-1, 1)

    docs = pd.read_json('./datasets/hateval_es_all.json', lines=True)
    textos_es_all = docs['text']
    Y_es_all = docs['klass']
    Y_es_all = Y_es_all.to_numpy().reshape(-1, 1)

    # --------------------------
    # Datos de entrenamiento
    # --------------------------
    textos_train = [textos_en_train, textos_es_train]
    Ys_train = [Y_en_train, Y_es_train]
    textos_test = [textos_en_test, textos_es_test]
    Ys_test = [Y_en_test, Y_es_test]

    # -----------------------------
    # Variables de los experimentos
    # -----------------------------
    pesados = ['TF', 'TF-IDF']
    n_gramas = [(1,1), (2, 2), (1,2)]
    preprocesamientos = [(normaliza_texto, None, None), (normaliza_texto, eliminar_stopwords, None), (normaliza_texto, eliminar_stopwords, aplicar_stemming)]
    initialization_functions = [xavier_initialization, normal_distribution_initialization]
    neuronas_ocultas = [64, 128, 256, 512, 1024]
    learning_rate = [0.01, 0.1, 0.5]
    batch_size = [16, 32, 64]
    epochs = 100
    salidas = 1

    nombres_datasets = ['EN', 'ES']
    nombres_n_gramas = ['Unigramas', 'Bigramas', 'Unigramas+Bigramas']
    nombres_preprocesamientos = ['Normalizacion', 'Normalizacion+Stopwords', 'Normalizacion+Stopwords+Stemming']
    nombres_inicializacion = ['Xavier', 'Normal distribution']

<h2>Selección aleatoria de experimentos</h2>

In [76]:
if EXPERIMENTAR:
    total_experimentos = []
    for x in product(pesados, 
                    zip(n_gramas, nombres_n_gramas), 
                    zip(preprocesamientos, nombres_preprocesamientos), 
                    zip(initialization_functions, nombres_inicializacion),
                    neuronas_ocultas,
                    learning_rate,
                    batch_size):
        total_experimentos.append(x)
    random.seed(RANDOM_STATE)
    experimentos = random.sample(total_experimentos, 150)

<h2>Ciclo principal</h2>

In [77]:
# --------------------------------------
# Realizar experimentos por cada Dataset
# --------------------------------------
if EXPERIMENTAR:
    n_modelos = 0

    for i in range(len(textos_train)):
        nombre_dataset = nombres_datasets[i]
        textos = textos_train[i]
        Y = Ys_train[i]
        X_test = textos_test[i]
        Y_test = Ys_test[i]
        for i_p in range(len(pesados)):
            pesado = pesados[i_p]
            for i_ngram in range(len(n_gramas)):
                nombre_grama = nombres_n_gramas[i_ngram]
                n_grama = n_gramas[i_ngram]
                for i_proc in range(len(preprocesamientos)):            
                    nombre_preprocesamiento = nombres_preprocesamientos[i_proc]
                    preprocesamiento = preprocesamientos[i_proc]
                    
                    #Agrupar los experimentos por procesamiento para optmizar el preprocesamiento 
                    experimentos_preprocesamientos = [e for e in experimentos if e[0] == pesado
                                                        and e[1][1] == nombre_grama
                                                        and e[2][1] == nombre_preprocesamiento]
                    print('Preprocesando textos...')
                    vec, X = tokenizar(textos, pesado, n_grama, preprocesamiento, nombre_dataset)
                    
                    for e in experimentos_preprocesamientos:                  
                        inicializacion = e[3][0]
                        nombre_inicializacion = e[3][1]
                        neuronas = e[4]
                        learning_rate = e[5]
                        batch = e[6]
                        diccionario = {
                                    'Dataset':nombre_dataset, 
                                    'Pesado':pesado,
                                    'N-gramas':nombre_grama,
                                    'Preprocesamiento':nombre_preprocesamiento,
                                    'Initialization':nombre_inicializacion,
                                    'Hidden-layer-size':neuronas,
                                    'Learning-rate':learning_rate,
                                    'Batch-size':batch,
                                    'Input-size':X.shape[1],
                                    'Output-size':salidas,
                                    'Activation':'Sigmoid'}
                        print('Características del modelo:')
                        print(diccionario)
                        modelo = MultiLayerPerceptron(num_entradas=X.shape[1], 
                                                    num_neuronas_ocultas=neuronas, 
                                                    num_salidas=salidas, 
                                                    inicialitazion_function=inicializacion, 
                                                    loss_function=loss_function_MSE,
                                                    activation_function=sigmoid,
                                                    activation_derivative=sigmoid_derivative,
                                                    epochs=epochs, 
                                                    batch_size=batch, 
                                                    learning_rate=learning_rate,
                                                    stop_error=0.001,
                                                    rango_epocas=10,
                                                    umbral_mejora=0.0005)
                        modelo.train(X.toarray(), Y)
                        print(f'{'█'*50} Resultados {'█'*50}')
                        a, f1, pr, rc = test_modelo(modelo, vec, X_test, Y_test)
                        diccionario.update({    
                            'Epochs':modelo.epochs,
                            'Accuracy':a,
                            'F1-score':f1,
                            'Precision':pr,
                            'Recall':rc
                        })
                        guardar_modelos((diccionario, modelo), f'Modelos{nombres_datasets[i]}.pkl')
                        graficar(diccionario, modelo)
                        n_modelos += 1
                        with open('DiccionarioModelos.txt', 'a') as file:
                            file.write(f'{n_modelos} {diccionario}\n')

<h1>Análisis de variables</h1>

In [ ]:
def cargar_modelos(filename, key = None, value = None):
    file = open(filename, 'rb')
    modelos = []
    while 1:
        try:
            tupla = pickle.load(file)
            if key is not None and value is not None:
                if (tupla[0][key] == value):
                    modelos.append((tupla[0], tupla[1].epochs_error))
            else:
                modelos.append((tupla[0], tupla[1].epochs_error))
                
        except EOFError:
            break     
    file.close()
    return modelos
    
def graficar_conjunto(listas_modelos, nombres_labels, titulo, caracteristicas, filename, epocas, opacidad=0.2):
    """
    Grafica un conjunto de redes neuronales en la misma grafica
    """
    colores = ['black', 'darkred', 'forestgreen', 'darkviolet', 'darkorange', 'royalblue']
    marcadores = ['solid', (0, (3, 1, 1, 1)), 'dotted', (0, (5, 1)), 'dashed', (0, (5, 10)), (5, (10, 3)), (0, (3, 5, 1, 5))]
    i = 0
    offset = len(colores) - len(listas_modelos)
    patches = []
    plt.figure(figsize=(16, 9))
    plt.title(titulo)
    plt.figtext(0.13, 0.03, caracteristicas)
    for l in listas_modelos:    
        color = colores[i + offset]
        for t in l:
            epocas_modelo = len(t[1])
            epocas_faltantes = 100 - epocas_modelo
            plt.plot(np.arange(epocas), t[1] + [None]*epocas_faltantes, color=color, linestyle=marcadores[i], linewidth=1, alpha=opacidad)
            if len(t[1]) < 100:
                plt.plot(len(t[1]) - 1, t[1][-1], 'x', color=color)
        patches.append(mpatches.Patch(color=color, label=nombres_labels[i]))
        i += 1
    plt.legend(handles=patches)
    plt.xlabel('Época')
    plt.ylabel('MSE')
    plt.grid(True)
    plt.savefig(filename)
    plt.close()

In [79]:
# --------------------------------------
# Filtrar por tipo de pesado
# --------------------------------------
pesado_TF = cargar_modelos('ModelosEN.pkl', 'Pesado', 'TF')
pesado_TF_IDF = cargar_modelos('ModelosEN.pkl', 'Pesado', 'TF-IDF')

graficar_conjunto(listas_modelos=[pesado_TF, pesado_TF_IDF], 
                  nombres_labels=['TF', 'TF-IDF'], 
                  titulo='Error por tipo de vectorización', 
                  caracteristicas='Dataset: EN', 
                  filename='./graficas/conjuntos/Pesado.png', 
                  epocas=100,
                  opacidad=0.5)

# --------------------------------------
# Filtrar por tipo de representación
# --------------------------------------
modelos = pesado_TF + pesado_TF_IDF
unigramas = [t for t in modelos if t[0]['N-gramas'] == 'Unigramas']
bigramas = [t for t in modelos if t[0]['N-gramas'] == 'Bigramas']
multigramas = [t for t in modelos if t[0]['N-gramas'] == 'Unigramas+Bigramas']

graficar_conjunto(listas_modelos=[unigramas, bigramas, multigramas], 
                  nombres_labels=['Unigramas', 'Bigramas', 'Unigramas y Bigramas'], 
                  titulo='Error por tipo de representación de términos', 
                  caracteristicas='Dataset: EN', 
                  filename='./graficas/conjuntos/NGramas.png', 
                  epocas=100,
                  opacidad=0.5)

# --------------------------------------
# Filtrar por tipo de preprocesamiento
# --------------------------------------
normalizacion = [t for t in modelos if t[0]['Preprocesamiento'] == 'Normalizacion']
norm_stopwords = [t for t in modelos if t[0]['Preprocesamiento'] == 'Normalizacion+Stopwords']
norm_stop_stemming = [t for t in modelos if t[0]['Preprocesamiento'] == 'Normalizacion+Stopwords+Stemming']

graficar_conjunto(listas_modelos=[normalizacion, norm_stopwords, norm_stop_stemming], 
                  nombres_labels=['Normalizacion', 'Normalizacion y remoción de stopwords', 'Normalizacion, remoción de stopwords y Stemming'], 
                  titulo='Error por tipo de preprocesamiento', 
                  caracteristicas='Dataset: EN', 
                  filename='./graficas/conjuntos/Preprocesamiento.png', 
                  epocas=100,
                  opacidad=0.5)

# --------------------------------------
# Filtrar por cantidad de neuronas
# --------------------------------------
n_64 = [t for t in modelos if t[0]['Hidden-layer-size'] == 64]
n_128 = [t for t in modelos if t[0]['Hidden-layer-size'] == 128]
n_256 = [t for t in modelos if t[0]['Hidden-layer-size'] == 256]
n_512 = [t for t in modelos if t[0]['Hidden-layer-size'] == 512]
n_1024 = [t for t in modelos if t[0]['Hidden-layer-size'] == 1024]

graficar_conjunto(listas_modelos=[n_64, n_128, n_256, n_512, n_1024], 
                  nombres_labels=['64', '128', '256', '512', '1024'], 
                  titulo='Error por tipo de preprocesamiento', 
                  caracteristicas='Dataset: EN', 
                  filename='./graficas/conjuntos/Neuronas.png', 
                  epocas=100,
                  opacidad=0.5)

KeyboardInterrupt: 

<h1>Mejores modelos</h1>

<h2>K-folds</h2>

In [1]:
# -----------------------------------------
# Obtener los 3 mejores modelos por dataset
# -----------------------------------------
def leer_diccionarios(filename):
    """
    Lee todos los diccionarios desde el archivo de texto y los regresa
    """
    file = open(filename, 'r')
    diccionarios = []
    for line in file.readlines():   
        _, d = line.split(' ', maxsplit=1)
        diccionarios.append(eval(d[:-1]))
    file.close()
    return diccionarios

def get_f1(d):
    return d['F1-score']

lista_diccionarios = leer_diccionarios('DiccionarioModelos.txt')
diccionarios_EN = lista_diccionarios[:150]
diccionarios_ES = lista_diccionarios[150:]
diccionarios_EN.sort(key=get_f1, reverse=True)
diccionarios_ES.sort(key=get_f1, reverse=True)
top3_EN = diccionarios_EN[:3]
top3_ES = diccionarios_ES[:3]
mejores_modelos = top3_EN + top3_ES

In [9]:
i = 0
for m in mejores_modelos:
    print(i)
    print(f'F1: {m['F1-score']:.3f}')
    print(f'Precision: {m['Precision']:.3f}')
    print(f'Recall: {m['Recall']:.3f}')
    print(f'Accuracy: {m['Accuracy']:.3f}')
    i += 1

0
F1: 0.732
Precision: 0.735
Recall: 0.740
Accuracy: 0.733
1
F1: 0.728
Precision: 0.728
Recall: 0.729
Accuracy: 0.734
2
F1: 0.726
Precision: 0.725
Recall: 0.726
Accuracy: 0.731
3
F1: 0.789
Precision: 0.790
Recall: 0.788
Accuracy: 0.792
4
F1: 0.785
Precision: 0.785
Recall: 0.785
Accuracy: 0.788
5
F1: 0.785
Precision: 0.785
Recall: 0.785
Accuracy: 0.788


In [ ]:
if EXPERIMENTAR:
    skf = StratifiedKFold(n_splits=5)
    n_modelos = 0
    accuracys = []
    precisions = []
    recalls = []
    f1_scores = []

    for d in mejores_modelos:
        #Buscar los parámetros a partir de la string
        nombre_grama = d['N-gramas']
        nombre_preprocesamiento = d['Preprocesamiento']
        nombre_inicializacion = d['Initialization']
        i_grama = nombres_n_gramas.index(nombre_grama)
        i_preprocesamiento = nombres_preprocesamientos.index(nombre_preprocesamiento)
        i_init = nombres_inicializacion.index(nombre_inicializacion)
        n_grama = n_gramas[i_grama]
        preprocesamiento = preprocesamientos[i_preprocesamiento]
        inicializacion = initialization_functions[i_init]

        if d['Dataset'] == 'EN':
            X_all = textos_en_all
            Y_all = Y_en_all
        elif d['Dataset'] == 'ES':
            X_all = textos_es_all
            Y_all = Y_es_all
            
        for k, (index_train, index_test) in enumerate(skf.split(X_all, Y_all), start=1):
            X_train_fold, X_test_fold = X_all[index_train], X_all[index_test]
            Y_train_fold, Y_test_fold = Y_all[index_train], Y_all[index_test]
            print(f"{'#'*50} k = {k} {'#'*50}")
            vec, X = tokenizar(X_train_fold, d['Pesado'], n_grama, preprocesamiento, d['Dataset'])
            diccionario = {
                'Dataset':d['Dataset'],
                'Pesado':d['Pesado'],
                'N-gramas':d['N-gramas'],
                'Preprocesamiento':d['Preprocesamiento'],
                'Initialization':d['Initialization'],
                'Hidden-layer-size':d['Hidden-layer-size'],
                'Learning-rate':d['Learning-rate'],
                'Batch-size':d['Batch-size'],
                'Input-size':X.shape[1],
                'Output-size':d['Output-size'],
                'Activation':d['Activation']
                }
            modelo = MultiLayerPerceptron(num_entradas=X.shape[1],
                                        num_neuronas_ocultas=d['Hidden-layer-size'], 
                                        num_salidas=d['Output-size'], 
                                        inicialitazion_function=inicializacion, 
                                        loss_function=loss_function_MSE,
                                        activation_function=sigmoid,
                                        activation_derivative=sigmoid_derivative,
                                        epochs=epochs, 
                                        batch_size=d['Batch-size'], 
                                        learning_rate=d['Learning-rate'],
                                        stop_error=0.001,
                                        rango_epocas=10,
                                        umbral_mejora=0.0005
                                        )
            modelo.train(X.toarray(), Y_train_fold)
            print(f'{'█'*50} Resultados {'█'*50}')
            a, f1, pr, rc = test_modelo(modelo, vec, X_test_fold, Y_test_fold)
            diccionario.update({
                'Epochs':modelo.epochs,
                'Accuracy':a,
                'F1-score':f1,
                'Precision':pr,
                'Recall':rc
            })
            accuracys.append(a)
            precisions.append(pr)
            recalls.append(rc)
            f1_scores.append(f1)
            guardar_modelos((diccionario, modelo), f'Modelos{d['Dataset']}_{n_modelos}.pkl')
            graficar(diccionario, modelo)
            with open(f'Diccionarios{d['Dataset']}_{n_modelos}.txt', 'a') as file:
                file.write(f'{n_modelos} {diccionario}\n')
        n_modelos += 1

<h2>Resultados</h2>

In [81]:
# ----------------------
# Promediar los K-folds
# ----------------------
nombres = !ls | grep DiccionariosE
accuracys = []
precisions = []
recalls = []
f1_scores = []
resultados = []

for n in nombres:
    lista_diccionarios = leer_diccionarios(n)
    for d in lista_diccionarios:
        accuracys.append(d['Accuracy'])
        precisions.append(d['Precision'])
        recalls.append(d['Recall'])
        f1_scores.append(d['F1-score'])
    accuracys = np.array(accuracys, dtype=np.float64)
    a = accuracys.mean()
    precisions = np.array(precisions, dtype=np.float64)
    pr = precisions.mean()
    recalls = np.array(recalls, dtype=np.float64)
    rc = recalls.mean()
    f1_scores = np.array(f1_scores, dtype=np.float64)
    f1 = f1_scores.mean()
    resultados.append((n[-5], a, pr, rc, f1))
    print(f'N: {n[-5]} {d}')
    print(f'F1-score: {f1}')
    print(f'Precision: {pr}')
    print(f'Recall: {rc}')
    print(f'Accuracy: {a}\n')
    accuracys = []
    precisions = []
    recalls = []
    f1_scores = []

N: 0 {'Dataset': 'EN', 'Pesado': 'TF-IDF', 'N-gramas': 'Unigramas+Bigramas', 'Preprocesamiento': 'Normalizacion', 'Initialization': 'Xavier', 'Hidden-layer-size': 64, 'Learning-rate': 0.1, 'Batch-size': 32, 'Input-size': 132217, 'Output-size': 1, 'Activation': 'Sigmoid', 'Epochs': 42, 'Accuracy': 0.638, 'F1-score': 0.637628932026395, 'Precision': 0.6636009071058024, 'Recall': 0.6595202638671802}
F1-score: 0.717871554397165
Precision: 0.7252178092955462
Recall: 0.7218599108135494
Accuracy: 0.7237

N: 1 {'Dataset': 'EN', 'Pesado': 'TF', 'N-gramas': 'Unigramas+Bigramas', 'Preprocesamiento': 'Normalizacion', 'Initialization': 'Xavier', 'Hidden-layer-size': 64, 'Learning-rate': 0.01, 'Batch-size': 32, 'Input-size': 132217, 'Output-size': 1, 'Activation': 'Sigmoid', 'Epochs': 81, 'Accuracy': 0.643, 'F1-score': 0.6417529419910709, 'Precision': 0.6463942307692307, 'Recall': 0.6499021574588015}
F1-score: 0.7121098824708614
Precision: 0.7298494944318876
Recall: 0.7102666978450026
Accuracy: 0.726